In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
from PIL import Image
import bm3d

In [2]:
def minMaxNormalized(img):
    return cv2.normalize(img, None, 0, 1, cv2.NORM_MINMAX,dtype=cv2.CV_32F)

In [3]:
def colorSeparator(image_rgb):
    r = image_rgb[:,:,0]
    g = image_rgb[:,:,1]
    b = image_rgb[:,:,2]
    return r, g, b

In [4]:
def high_boost(img, low_contrast, A):
    return ((A-1)*low_contrast + img)

In [5]:
# return red, green and blue freq. domain
def to_frequencyDomainGray(gray_img):
    normalized     = minMaxNormalized(gray_img)
    norm_fft       = np.fft.fft2(normalized)
    norm_fft_shift = np.fft.fftshift(norm_fft)
    return norm_fft_shift

def to_frequencyDomain(image): 
    r,g,b = colorSeparator(image)
    r_fftshft = to_frequencyDomainGray(r)
    b_fftshft = to_frequencyDomainGray(b)
    g_fftshft = to_frequencyDomainGray(g)
    return r_fftshft, b_fftshft, g_fftshft

def magnitudeLog(img):
    mag = np.sqrt(img.real ** 2 + img.imag ** 2)
    return np.log1p(mag)

In [6]:
def to_spatialDomain(image):
    image_ishift = np.fft.ifftshift(image)
    image_ifft   = np.fft.ifft2(image_ishift)
    image_real   = np.abs(image_ifft)
    # denormlized  = image * 255
    return image_real

In [7]:
def reduceSaltAndPeperNoise(image, kernel=3):
    median_filtered = cv2.medianBlur(image, kernel)
    return median_filtered

In [19]:
#use bm3d method. before use this function make sure install package. (pip3 install bm3d)
def reduceGaussianNoise(image, sigma=20/255):
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    R, G, B = cv2.split(image_rgb)

    R_denoised = bm3d.bm3d(R / 255.0, sigma) * 255
    G_denoised = bm3d.bm3d(G / 255.0, sigma) * 255
    B_denoised = bm3d.bm3d(B / 255.0, sigma) * 255

    image_denoised = cv2.merge((R_denoised, G_denoised, B_denoised))
    image_denoised = np.clip(image_denoised, 0, 255).astype(np.uint8)

    image_denoised = cv2.cvtColor(image_denoised, cv2.COLOR_RGB2BGR)
    
    return image_denoised

In [9]:
#periodic noise...

In [10]:
# for one type of periodic noise. which type?
def NotchFilter(shape, radius, center_x, center_y):
    dimention1, dimention2 = shape
    mask = np.zeros((dimention1, dimention2, 2))

    for i in range(0, dimention1):
        for item in range(0, dimention2):
            radius_hat1 = np.sqrt((i - center_x) ** 2 + (item - center_y) ** 2)
            radius_hat2 = np.sqrt((i - dimention1 + center_x) ** 2 + (item - dimention2 + center_y) ** 2)

            if radius_hat1 <= radius or radius_hat2 <= radius:
                mask[i, item] = 0.0
            else:
                mask[i, item] = 1.0
    return mask

In [11]:
# maybe we need this function. negative of above function. return just periodic noise
def negetiveNotchFilter(shape, radius, center_x, center_y):
    dimention1, dimention2 = shape
    mask = np.zeros((dimention1, dimention2, 2))

    for i in range(0, dimention1):
        for item in range(0, dimention2):
            radius_hat1 = np.sqrt((i - center_x) ** 2 + (item - center_y) ** 2)
            radius_hat2 = np.sqrt((i - dimention1 + center_x) ** 2 + (item - dimention2 + center_y) ** 2)

            if radius_hat1 <= radius or radius_hat2 <= radius:
                mask[i, item] = 1.0
            else:
                mask[i, item] = 0.0
    return mask

In [12]:
# maybe we need this function
def psnr(original_image, reconstructed_image):
    original_image      = original_image.astype(np.float64)
    reconstructed_image = reconstructed_image.astype(np.float64)

    mse = np.mean((original_image - reconstructed_image) ** 2)

    if mse == 0:
        return float('inf')

    max_pixel_value = 255 
    psnr = 10 * np.log10((max_pixel_value ** 2) / mse)
    return psnr

In [13]:
#input: BGR image
def equalizeHistForColorImages (img):
    img_yuv = cv2.cvtColor(img, cv2.COLOR_BGR2YUV)
    img_yuv[:, :, 0] = cv2.equalizeHist(img_yuv[:, :, 0])
    return cv2.cvtColor(img_yuv, cv2.COLOR_YUV2BGR)